In [6]:
import nltk
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import json
from time import time
from collections import defaultdict
import math
import os
import wikipediaapi as wiki_api

# Δημιουργία κλάσης WikipediaReader
class WikipediaReader():
    def __init__(self, dir="articles"):
        # Αρχικοποίηση της κλάσης, ορίζουμε τον φάκελο αποθήκευσης άρθρων και τον πυρήνα Wikipedia API.
        self.pages = set()
        self.article_path = os.path.join("./", dir)
        self.wiki = wiki_api.Wikipedia(
            language='en',  # Ορίζουμε τη γλώσσα Wikipedia σε Αγγλικά.
            extract_format=wiki_api.ExtractFormat.WIKI,  # Ορίζουμε τη μορφή εξαγωγής σε κείμενο Wiki.
            user_agent="MyWikipediaReaderApp/1.0 (myemail@example.com)"  # Πρόσθεσε user_agent
        )
        # Δημιουργία φακέλου αν δεν υπάρχει
        try:
            os.mkdir(self.article_path)
        except Exception as e:
            pass

    # Μέθοδος για να μετατρέπουμε τον τίτλο του άρθρου σε μορφή συμβατή με αρχεία (π.χ., αντικατάσταση κενών).
    def _get_page_title(self, article):
        return re.sub(r'\s+', '_', article)

    # Μέθοδος για προσθήκη ενός άρθρου
    def add_article(self, article):
        try:
            # Παίρνουμε τη σελίδα από τη Wikipedia με τον δεδομένο τίτλο
            page = self.wiki.page(self._get_page_title(article))
            # Ελέγχουμε αν η σελίδα υπάρχει
            if page.exists():
                self.pages.add(page)  # Προσθέτουμε τη σελίδα στο σύνολο `pages`
                return page
        except Exception as e:
            print(e)

    # Μέθοδος που επιστρέφει τη λίστα των σελίδων που έχουν προστεθεί
    def list(self):
        return self.pages

    # Μέθοδος για λήψη και αποθήκευση των άρθρων σε αρχεία
    def process(self, update=False):
        for page in self.pages:
            # Δημιουργία έγκυρου ονόματος αρχείου για την αποθήκευση
            filename = re.sub('\s+', '_', f'{page.title}')
            filename = re.sub(r'[\\/*?:"<>|!]', '', filename)
            file_path = os.path.join(self.article_path, f'{filename}.txt')
            # Αν επιτρέπεται το update ή το αρχείο δεν υπάρχει, κατεβάζουμε και αποθηκεύουμε το άρθρο
            if update or not os.path.exists(file_path):
                print(f'Downloading {page.title} ...')
                content = page.text
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.write(content)
            else:
                print(f'Not updating {page.title} ...')

    # Μέθοδος που κάνει "crawl" τα συνδεδεμένα άρθρα με δεδομένο βάθος και μέγιστο αριθμό άρθρων
    def crawl_pages(self, article, depth=3, total_number=1000):
        print(f'Crawl {total_number} :: {article}')
        page = self.add_article(article)  # Προσθέτουμε το αρχικό άρθρο στη λίστα
        childs = set()  # Δημιουργούμε σύνολο για τα συνδεδεμένα άρθρα

        if page:
            for child in page.links.keys():
                # Αν δεν έχουμε φτάσει το όριο άρθρων, προσθέτουμε συνδεδεμένο άρθρο
                if len(self.pages) < total_number:
                    print(f'Add article {len(self.pages)}/{total_number} {child}')
                    self.add_article(child)
                    childs.add(child)

        # Μειώνουμε το βάθος και συνεχίζουμε το crawl για κάθε συνδεδεμένο άρθρο
        depth -= 1
        if depth > 0:
            for child in sorted(childs):
                if len(self.pages) < total_number:
                    self.crawl_pages(child, depth, len(self.pages))

    # Μέθοδος για λήψη των κατηγοριών του άρθρου, με φιλτράρισμα περιττών κατηγοριών
    def get_categories(self, title):
        page = self.add_article(title)
        if page:
            # Φιλτράρισμα μη σχετικών κατηγοριών
            if (list(page.categories.keys())) and (len(list(page.categories.keys())) > 0):
                categories = [c.replace('Category:', '').lower() for c in list(page.categories.keys())
                              if c.lower().find('articles') == -1
                              and c.lower().find('pages') == -1
                              and c.lower().find('wikipedia') == -1
                              and c.lower().find('cs1') == -1
                              and c.lower().find('webarchive') == -1
                              and c.lower().find('dmy dates') == -1
                              and c.lower().find('short description') == -1
                              and c.lower().find('commons category') == -1]
                # Επιστρέφει λεξικό με κάθε κατηγορία και την τιμή 1
                return dict.fromkeys(categories, 1)
        return {}
    
    ############################################################################################################
    
    # Εισαγωγή και δημιουργία ενός αντικειμένου WikipediaReader
reader = WikipediaReader(dir="my_articles")

# 1. Προσθήκη ενός άρθρου, π.χ., "Python (programming language)"
article = reader.add_article("Python (programming language)")

# 2. Καταγραφή όλων των άρθρων που έχουν προστεθεί
print("Προστέθηκαν τα ακόλουθα άρθρα:")
for page in reader.list():
    print(page.title)

# 3. Λήψη και αποθήκευση των άρθρων σε αρχεία
reader.process(update=False)  # Αν το αρχείο υπάρχει ήδη, δεν θα το ενημερώσει

# 4. Κάνοντας crawl σε συνδεδεμένα άρθρα για την επέκταση της βάσης δεδομένων
# Κάνουμε crawl στο άρθρο "Python (programming language)" μέχρι βάθος 2 και με όριο 50 άρθρα.
reader.crawl_pages("Python (programming language)", depth=2, total_number=50)

# 5. Ενημέρωση για όλα τα νέα άρθρα που προστέθηκαν
reader.process(update=True)

# 6. Ανάκτηση των κατηγοριών για ένα συγκεκριμένο άρθρο, π.χ., "Python (programming language)"
categories = reader.get_categories("Python (programming language)")
print("Κατηγορίες του άρθρου 'Python (programming language)':", categories)


# Κατεβάζει τα απαραίτητα δεδομένα αν δεν υπάρχουν
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

class WikipediaCorpus(PlaintextCorpusReader):
    def __init__(self, root, fileids):
        super().__init__(root, fileids)
        self.lemmatizer = WordNetLemmatizer()  # Αρχικοποίηση του lemmatizer
        self.stop_words = set(stopwords.words('english'))  # Χρήση αγγλικών stop words
        self.inverted_index = self.create_inverted_index()  # Δημιουργία αντεστραμμένου ευρετηρίου

    def clean_text(self, text):
        # Καθαρισμός κειμένου: αφαιρεί σημεία στίξης, κάνει tokenization,αφαιρεί stop words και εφαρμόζει lemmatization. 
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()  # Αφαίρεση μη αλφαριθμητικών χαρακτήρων και μετατροπή σε πεζά
        tokens = word_tokenize(text)  # Tokenization
        tokens = [word for word in tokens if word not in self.stop_words]  # Αφαίρεση stop words
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]  # Εφαρμογή lemmatization
        return tokens

    def preprocess_documents(self):
        # Καθαρισμός όλων των εγγράφων του corpus. 
        cleaned_docs = {}
        for fileid in self.fileids():  # Διατρέχει όλα τα αρχεία του corpus
            raw_text = self.raw(fileids=fileid)  # Παίρνει το ακατέργαστο κείμενο
            cleaned_docs[fileid] = self.clean_text(raw_text)  # Καθαρισμός κειμένου
        return cleaned_docs
    
    def vocab(self):
        # Υπολογισμός συχνότητας λέξεων στο καθαρισμένο σύνολο δεδομένων. 
        return nltk.FreqDist(word for fileid in self.fileids() for word in self.clean_text(self.raw(fileid)))
    
    def max_words(self):
        # Υπολογισμός μέγιστου αριθμού λέξεων σε έγγραφο.  
        return max(len(self.clean_text(self.raw(fileid))) for fileid in self.fileids())
    
    def create_inverted_index(self):
        # Δημιουργία αντεστραμμένου ευρετηρίου. 
        inverted_index = defaultdict(list)
        for fileid in self.fileids():  # Διατρέχει όλα τα αρχεία
            tokens = self.clean_text(self.raw(fileids=fileid))  # Καθαρισμός κειμένου
            for token in set(tokens):  # Για κάθε μοναδικό token
                inverted_index[token].append(fileid)  # Προσθήκη στο ευρετήριο
        return inverted_index

    def query_processing(self, query):
        # Εκτέλεση αναζήτησης. Υποστηρίζει τελεστές AND, OR, NOT. 
        tokens = re.findall(r'\w+|AND|OR|NOT', query)  # Διαχωρισμός του ερωτήματος σε tokens
        result_docs = set(self.fileids())  # Αρχικοποίηση με όλα τα έγγραφα
        current_op = 'OR'  # Προκαθορισμένος τελεστής
        exclude_docs = set()

        for token in tokens:
            if token in {'AND', 'OR', 'NOT'}:  # Αν είναι τελεστής, αποθηκεύεται
                current_op = token
            else:
                token_docs = set(self.inverted_index.get(token, []))  # Ανάκτηση σχετικών εγγράφων για το token
                if current_op == 'AND':
                    result_docs &= token_docs
                elif current_op == 'OR':
                    result_docs |= token_docs  
                elif current_op == 'NOT':
                    exclude_docs |= token_docs  # Προσθήκη στον αποκλεισμό

        result_docs -= exclude_docs  # Αποκλεισμός εγγράφων
        return list(result_docs)

    def compute_tf_idf(self, query):
        # Υπολογισμός TF-IDF και επιστροφή εγγράφων με βάση το σκορ συσχέτισης. 
        N = len(self.fileids())  # Συνολικός αριθμός εγγράφων
        query_tokens = self.clean_text(query)  # Καθαρισμός του ερωτήματος
        scores = defaultdict(float)

        for token in query_tokens:
            if token in self.inverted_index:
                df = len(self.inverted_index[token])  # Document frequency
                idf = math.log(N / (1 + df))  # Υπολογισμός IDF
                for doc in self.inverted_index[token]:
                    tf = self.raw(fileids=doc).lower().split().count(token) / len(self.raw(fileids=doc).split())
                    scores[doc] += tf * idf  # Υπολογισμός TF-IDF

        return sorted(scores.items(), key=lambda x: x[1], reverse=True)

    def vector_space_model(self, query):
        # Υλοποίηση Vector Space Model (VSM) με χρήση cosine similarity. 
        N = len(self.fileids())  # Συνολικός αριθμός εγγράφων
        query_tokens = self.clean_text(query)  # Καθαρισμός του ερωτήματος
        query_vector = defaultdict(float)
        doc_vectors = {doc: defaultdict(float) for doc in self.fileids()}  # Δημιουργία διανυσμάτων για κάθε έγγραφο

        # Υπολογισμός TF-IDF για το ερώτημα
        for token in query_tokens:
            if token in self.inverted_index:
                df = len(self.inverted_index[token])  # Συχνότητα εγγράφων για το token
                idf = math.log(N / (1 + df))
                query_vector[token] = idf  # Προσθήκη στο διάνυσμα του ερωτήματος
                for doc in self.inverted_index[token]:
                    tf = self.raw(fileids=doc).lower().split().count(token) / len(self.raw(fileids=doc).split())
                    doc_vectors[doc][token] = tf * idf  # Υπολογισμός TF-IDF για κάθε έγγραφο

        # Υπολογισμός cosine similarity
        def cosine_similarity(vec1, vec2):
            dot_product = sum(vec1[token] * vec2.get(token, 0) for token in vec1)  # Εσωτερικό γινόμενο
            norm1 = math.sqrt(sum(v ** 2 for v in vec1.values()))  # Μέτρο του πρώτου διανύσματος
            norm2 = math.sqrt(sum(v ** 2 for v in vec2.values()))  # Μέτρο του δεύτερου διανύσματος
            return dot_product / (norm1 * norm2) if norm1 and norm2 else 0  # Υπολογισμός cosine similarity

        scores = {doc: cosine_similarity(query_vector, doc_vectors[doc]) for doc in self.fileids()}
        return sorted(scores.items(), key=lambda x: x[1], reverse=True)

    def probabilistic_retrieval(self, query, k1=1.5, b=0.75):
        # Υλοποίηση Probabilistic Retrieval Model με χρήση BM25. 
        N = len(self.fileids())  # Συνολικός αριθμός εγγράφων
        avg_doc_length = sum(len(self.clean_text(self.raw(fileids=doc))) for doc in self.fileids()) / N  # Μέσο μήκος εγγράφου
        query_tokens = self.clean_text(query)  # Καθαρισμός του ερωτήματος
        scores = defaultdict(float)

        for token in query_tokens:
            if token in self.inverted_index:
                df = len(self.inverted_index[token])  # Συχνότητα εγγράφων
                idf = math.log((N - df + 0.5) / (df + 0.5) + 1)  # Υπολογισμός IDF
                for doc in self.inverted_index[token]:
                    doc_length = len(self.clean_text(self.raw(fileids=doc)))  # Μήκος εγγράφου
                    tf = self.raw(fileids=doc).lower().split().count(token)  # Συχνότητα λέξης
                    score = idf * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (doc_length / avg_doc_length))))  # Υπολογισμός BM25
                    scores[doc] += score  # Προσθήκη στο σκορ

        return sorted(scores.items(), key=lambda x: x[1], reverse=True)

    def describe(self, fileids=None):
        # Παροχή στατιστικών για το corpus. 
        started = time()
        stats = {
            'files': len(self.fileids()),  # Αριθμός αρχείων
            'paras': len(self.paras()),  # Αριθμός παραγράφων
            'sents': len(self.sents()),  # Αριθμός προτάσεων
            'words': len(self.words()),  # Αριθμός λέξεων
            'vocab': len(self.vocab()),  # Μέγεθος λεξιλογίου
            'time': time() - started  # Χρόνος επεξεργασίας
        }
        return stats


# Εφαρμογή του corpus και προεπεξεργασία
corpus = WikipediaCorpus('my_articles', '.*\.txt')

# Περιγραφή του αρχικού συνόλου δεδομένων
stats = corpus.describe()
print("\nΣτατιστικά ανάλυσης για το σύνολο κειμένων:")
print(stats)
query = input("\nΔώσε ερώτημα για αναζήτηση: ")

# Εκτέλεση αναζήτησης με TF-IDF
tf_idf_results = corpus.compute_tf_idf(query)
print("\nΑποτελέσματα TF-IDF αναζήτησης:")
for doc, score in tf_idf_results:
    print(f"{doc}: {score}")
    
algo = input("\nΔώσε Αλγόριθμο ανάκτησης(Boolean, VSM, BM25): ")

if algo == 'Boolean':
    # Εκτέλεση ερωτήματος Boolean
    result_docs = corpus.query_processing(query)
    print("\nΑποτελέσματα Boolean αναζήτησης:")
    print(result_docs)
elif algo == 'VSM':
    # Εκτέλεση αναζήτησης με Vector Space Model
    vsm_results = corpus.vector_space_model(query)
    print("\nΑποτελέσματα Vector Space Model αναζήτησης:")
    for doc, score in vsm_results:
        print(f"{doc}: {score}")
elif algo == 'BM25':
    # Εκτέλεση αναζήτησης με Probabilistic Retrieval Model (BM25)
    bm25_results = corpus.probabilistic_retrieval(query)
    print("\nΑποτελέσματα Probabilistic Retrieval Model (BM25) αναζήτησης:")
    for doc, score in bm25_results:
        print(f"{doc}: {score}")
else:
    print("Λάθος αλγόριθμος ανάκτησης!")


Προστέθηκαν τα ακόλουθα άρθρα:
Python (programming language)
Crawl 50 :: Python (programming language)
Add article 2/50 "Hello, World!" program
Add article 3/50 3ds Max
Add article 4/50 ?:
Add article 5/50 ABC (programming language)
Add article 6/50 ADMB
Add article 7/50 ALGOL
Add article 8/50 ALGOL 68
Add article 9/50 API
Add article 10/50 APL (programming language)
Add article 11/50 ATmega
Add article 12/50 AVR microcontrollers
Add article 13/50 Abaqus
Add article 14/50 Academic Free License
Add article 15/50 Academic conference
Add article 16/50 Ada (programming language)
Add article 17/50 Advanced Simulation Library
Add article 18/50 Ahead-of-time compilation
Add article 19/50 Alex Martelli
Add article 20/50 Algebra
Add article 21/50 Alternative terms for free software
Add article 22/50 Amazon (company)
Add article 23/50 AmigaOS 4
Add article 24/50 Amoeba (operating system)
Add article 25/50 Anaconda (installer)
Add article 26/50 Analyse-it
Add article 27/50 Android (operating syst

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Στατιστικά ανάλυσης για το σύνολο κειμένων:
{'files': 48, 'paras': 2352, 'sents': 8246, 'words': 180393, 'vocab': 12650, 'time': 0.9748818874359131}



Δώσε ερώτημα για αναζήτηση:  machine AND python



Αποτελέσματα TF-IDF αναζήτησης:
Python_(programming_language).txt: 0.022585888285353548
Alex_Martelli.txt: 0.016612734080165125
Astropy.txt: 0.015065804854245716
Asynchronous_Server_Gateway_Interface.txt: 0.008588017062008653
Ahead-of-time_compilation.txt: 0.0058084959264799885
Assembly_language.txt: 0.003190662150715329
Anonymous_function.txt: 0.0027288827706926605
Amoeba_(operating_system).txt: 0.0020665921207268013
Artificial_intelligence.txt: 0.001981564740445601
Array_slicing.txt: 0.001718941109763414
Arbitrary-precision_arithmetic.txt: 0.00166657782537821
Anaconda_(installer).txt: 0.0016063467139273825
Ternary_conditional_operator.txt: 0.0007562775441804495
Apache_Groovy.txt: 0.0007212791395955005
Abaqus.txt: 0.0007053756423573742
Aspect-oriented_programming.txt: 0.0006838547070894721
APL_(programming_language).txt: 0.0006367911626641666
Assertion_(software_development).txt: 0.0006285406193752595
ADMB.txt: 0.0005549617138190114
Assignment_(computer_science).txt: 0.00054020944254


Δώσε Αλγόριθμο ανάκτησης(Boolean, VSM, BM25):  Boolean



Αποτελέσματα Boolean αναζήτησης:
['Astropy.txt', 'Array_slicing.txt', 'Aspect-oriented_programming.txt', 'Abaqus.txt', 'ABC_(programming_language).txt', 'Anaconda_(installer).txt', 'Apache_Groovy.txt', 'Assertion_(software_development).txt', 'Python_(programming_language).txt', 'Ternary_conditional_operator.txt', 'Arbitrary-precision_arithmetic.txt', 'Apache_HTTP_Server.txt', 'ArcGIS.txt', 'Asynchronous_Server_Gateway_Interface.txt', 'Assignment_(computer_science).txt', 'Associative_array.txt', 'Alex_Martelli.txt', 'AmigaOS_4.txt', 'Artificial_intelligence.txt', 'Hello,_World_program.txt', 'Anonymous_function.txt', 'Amoeba_(operating_system).txt']
